In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly 

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.ensemble import RandomForestClassifierr
from sklearn.model_selection import train_test_split0
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.metrics import classification_report, accuracy_score

from joblib import Parallel, delayed

from time import time

import optuna
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history
-  
import os

# pd.set_option('display.max_colwidth', None)
# pd.options.display.max_columns = None

c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
# dataset_file = 'competencia_01.csv'

base_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe.csv'


ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202104
mes_test = 202106

# agregue sus semillas
semillas = [540079, 250829, 314299, 302111, 801007]

data = pd.read_csv(dataset_path + dataset_file)

In [5]:
df=data.copy()
df.shape

(981946, 155)

In [13]:
df[["ctarjeta_master"]]

,ctarjeta_master
0,1
1,1
2,1
3,1
4,1
...,...
981941,1
981942,1
981943,1
981944,1


In [4]:
# mes de entrenamiento
train= df[df['foto_mes'] == mes_train]
predecir = df[df['foto_mes'] == mes_test]

In [6]:
## relleno todo con 0 de momento
#train.fillna(0, inplace=True, downcast= "infer")

C:\Users\Federico\AppData\Local\Temp\ipykernel_14972\2177770977.py:2: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  train.fillna(0, inplace=True, downcast= "infer")
C:\Users\Federico\AppData\Local\Temp\ipykernel_14972\2177770977.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.fillna(0, inplace=True, downcast= "infer")


# Cuales son las variables mas importantes para un arbol de decision?

In [26]:
# Separar las características y el target
X = train.drop(columns=['clase_ternaria'])  # Variables predictoras
y = train['clase_ternaria']  # Variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo Random Forest con parámetros generales
rf_model = DecisionTreeClassifier(
    criterion= 'gini',       # criterio
    max_depth=5,            # Profundidad máxima de cada árbol
    min_samples_split=10,   # Mínima cantidad de muestras para dividir un nodo
    min_samples_leaf=80,     # Mínima cantidad de muestras en una hoja
    random_state=semillas[0]
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Obtener las variables más importantes
importances = rf_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame con las importancias de cada variable
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)



Accuracy: 0.9869583765007008
Classification Report:


c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      BAJA+1       0.00      0.00      0.00       204
      BAJA+2       0.00      0.00      0.00       224
    CONTINUA       0.99      1.00      0.99     32390

    accuracy                           0.99     32818
   macro avg       0.33      0.33      0.33     32818
weighted avg       0.97      0.99      0.98     32818



c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [27]:
importance_df.head(30)

,Feature,Importance
107,ctrx_quarter,0.359070
22,mcuentas_saldo,0.153203
111,Master_status,0.152655
33,mprestamos_personales,0.149380
133,Visa_status,0.057169
16,mcuenta_corriente,0.053803
18,mcaja_ahorro,0.023468
127,Master_fechaalta,0.014814
51,cpayroll_trx,0.011745
11,mpasivos_margen,0.009328


## Optimizacion de parametros

In [19]:
def ganancia(model, X, y, prop=1, threshold=0.025):

  class_index = np.where(model.classes_ == 1)[0][0]
  y_hat = model.predict_proba(X)

  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == 1 else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop

In [10]:
sss_opt = ShuffleSplit(n_splits=5, test_size=0.3, random_state=semillas[1])

def objective(trial, X, y, sss):
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 2, 20)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 150)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 150)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 150)

  def train_and_evaluate(train_index, test_index, X, y):
    m = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_leaf_nodes=max_leaf_nodes,
        random_state=semillas[0],
    )
    m.fit(X.iloc[train_index],y.iloc[train_index])
    ganancia_value = ganancia(m, X.iloc[test_index], y.iloc[test_index], prop=0.3)
    return ganancia_value

  results = Parallel(n_jobs=-1)(
      delayed(train_and_evaluate)(train_index, test_index, X, y)
      for train_index, test_index in sss.split(X)
  )

  return np.mean(results)sss_opt = ShuffleSplit(n_splits=5, test_size=0.3, random_state=semillas[1])

def objective(trial, X, y, sss):
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 2, 20)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 150)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 150)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 150)

  def train_and_evaluate(train_index, test_index, X, y):
    m = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_leaf_nodes=max_leaf_nodes,
        random_state=semillas[0],
    )
    m.fit(X.iloc[train_index],y.iloc[train_index])
    ganancia_value = ganancia(m, X.iloc[test_index], y.iloc[test_index], prop=0.3)
    return ganancia_value

  results = Parallel(n_jobs=-1)(
      delayed(train_and_evaluate)(train_index, test_index, X, y)
      for train_index, test_index in sss.split(X)
  )

  return np.mean(results)

#storage_name = "sqlite:///G:/DM EyF/optimization_tree.db"
study_name = "exp_101_decision-tree-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    #storage=storage_name,
    #load_if_exists=True,
)

[I 2024-09-11 22:58:49,560] A new study created in memory with name: exp_101_decision-tree-opt


In [44]:
study.optimize(lambda trial: objective(trial, X, y, sss_opt), n_trials=50)

[I 2024-09-10 23:34:14,084] Trial 0 finished with value: 114856000.0 and parameters: {'criterion': 'gini', 'max_depth': 15, 'min_samples_split': 51, 'min_samples_leaf': 94, 'max_leaf_nodes': 23}. Best is trial 0 with value: 114856000.0.
[I 2024-09-10 23:34:18,751] Trial 1 finished with value: 106339333.33333334 and parameters: {'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 64, 'min_samples_leaf': 47, 'max_leaf_nodes': 86}. Best is trial 0 with value: 114856000.0.
[I 2024-09-10 23:34:22,303] Trial 2 finished with value: 103954666.66666669 and parameters: {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 111, 'min_samples_leaf': 31, 'max_leaf_nodes': 11}. Best is trial 0 with value: 114856000.0.
[I 2024-09-10 23:34:27,101] Trial 3 finished with value: 113143333.33333334 and parameters: {'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 65, 'min_samples_leaf': 13, 'max_leaf_nodes': 143}. Best is trial 0 with value: 114856000.0.
[I 2024-09-10 23:34:34,827

In [45]:
optuna.visualization.plot_optimization_history(study)

In [46]:
plot_param_importances(study)

In [47]:
plot_slice(study)

In [48]:
best_trial = study.best_trial
best_model_params = best_trial.params
print("Mejor modelo:", best_model_params)

Mejor modelo: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_split': 79, 'min_samples_leaf': 86, 'max_leaf_nodes': 31}


In [30]:
# Definir el modelo arbol de decision con los mejores parametros que encontre
rf_model = DecisionTreeClassifier(
    criterion= 'entropy',       # criterio
    max_depth=8,            # Profundidad máxima de cada árbol
    min_samples_split=700,   
    min_samples_leaf=86,
    max_leaf_nodes= 31,    
    random_state=semillas[0]
)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir sobre el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Obtener las variables más importantes
importances = rf_model.feature_importances_
feature_names = X.columns

# Crear un DataFrame con las importancias de cada variable
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

Accuracy: 0.9869583765007008
Classification Report:


c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      BAJA+1       0.00      0.00      0.00       204
      BAJA+2       0.00      0.00      0.00       224
    CONTINUA       0.99      1.00      0.99     32390

    accuracy                           0.99     32818
   macro avg       0.33      0.33      0.33     32818
weighted avg       0.97      0.99      0.98     32818



c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Federico\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
importance_df.head(30)

,Feature,Importance
107,ctrx_quarter,0.467382
52,mpayroll,0.131074
22,mcuentas_saldo,0.093377
33,mprestamos_personales,0.071196
11,mpasivos_margen,0.058927
29,ctarjeta_master,0.050536
137,Visa_msaldototal,0.026079
27,ctarjeta_visa_transacciones,0.017635
8,mrentabilidad_annual,0.017540
16,mcuenta_corriente,0.015618


## Ahora quiero predecir sobre junio, para eso entreno el modelo con todos los datos de abril 

In [32]:
# Entrenar el modelo con todos los datos de abril
rf_model.fit(X, y)

DecisionTreeClassifier(criterion='entropy', max_depth=8, max_leaf_nodes=31,
                       min_samples_leaf=86, min_samples_split=700,
                       random_state=540079)

In [33]:
X_predecir = predecir.drop(columns=['clase_ternaria'])  # Variables predictoras

In [34]:
y_pred = rf_model.predict_proba(X_predecir)

In [35]:
y_pred

array([[2.26250031e-04, 3.14236155e-04, 9.99459514e-01],
       [1.06882290e-03, 1.48705795e-03, 9.97444119e-01],
       [1.06882290e-03, 1.48705795e-03, 9.97444119e-01],
       ...,
       [1.06882290e-03, 1.48705795e-03, 9.97444119e-01],
       [9.91831972e-03, 7.00116686e-03, 9.83080513e-01],
       [2.26250031e-04, 3.14236155e-04, 9.99459514e-01]])

In [36]:
import os
# Convertir las predicciones a un DataFrame para fácil manejo
prediccion_df = pd.DataFrame(y_pred, columns=rf_model.classes_)

# Agregar la probabilidad de 'BAJA+2' al dataframe de aplicación
predecir['prob_baja2'] = prediccion_df['BAJA+2']

# Crear la columna 'Predicted' donde la probabilidad de 'BAJA+2' sea mayor a 1/40
predecir['Predicted'] = (predecir['prob_baja2'] > 1 / 40).astype(int)

# Generar el archivo para Kaggles
# Crear el directorio si no existe
os.makedirs("./exp/KA2001", exist_ok=True)

# Seleccionar solo los campos necesarios para Kaggle
kaggle_submission = predecir[['numero_de_cliente', 'Predicted']]

# Guardar el archivo
kaggle_submission.to_csv("./exp/KA2001/K101_003.csv", index=False, sep=",")

C:\Users\Federico\AppData\Local\Temp\ipykernel_14972\2009958921.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predecir['prob_baja2'] = prediccion_df['BAJA+2']
C:\Users\Federico\AppData\Local\Temp\ipykernel_14972\2009958921.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predecir['Predicted'] = (predecir['prob_baja2'] > 1 / 40).astype(int)


In [41]:
predecir[predecir['Predicted']==1]

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,clase_binaria,prob_baja,Predicted
23,250892233,202106,1,0,0,69,325,898.92,-4775.41,315.37,...,NaN,314.0,NaN,NaN,NaN,0.00,NaN,1,0.026667,1
57,252966056,202106,1,0,0,56,218,1924.55,31626.42,4114.42,...,0.00,8661.0,27764.42,8.0,0.0,5313.69,NaN,1,0.060870,1
87,255614475,202106,1,0,0,65,283,5732.54,41402.14,746.84,...,0.00,6128.0,12695.07,7.0,0.0,656.88,NaN,1,0.060694,1
158,259824701,202106,1,0,0,56,79,1016.80,-1740.48,737.50,...,0.00,2347.0,5728.81,4.0,0.0,480.93,NaN,1,0.060870,1
218,264937841,202106,1,0,0,49,159,-74.32,7306.47,3147.26,...,0.00,4816.0,1489.47,4.0,0.0,7002.81,NaN,1,0.083984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164659,329821777,202106,1,0,0,48,265,5945.80,58356.22,3397.81,...,1.16,7909.0,7184.12,4.0,0.0,1653.93,NaN,1,0.026667,1
164677,331819759,202106,1,0,0,72,296,2730.62,30327.77,3026.83,...,0.00,8990.0,52955.81,19.0,0.0,2744.82,NaN,1,0.083984,1
164695,332496302,202106,1,0,0,42,177,-498.60,-2193.25,797.59,...,34.39,5344.0,36799.83,19.0,0.0,6439.77,NaN,1,0.066007,1
164749,339121872,202106,1,0,0,55,378,547.26,28134.24,27.84,...,0.00,9365.0,2457.43,1.0,0.0,117.30,NaN,1,0.083172,1


In [ ]:
# me da tan mal que empizo de nuevo, unifico las clases baja+1 y baja+2

In [16]:
df['clase_binaria'] = df['clase_ternaria'].apply(lambda x: 0 if x == 'CONTINUA' else 1)

train= df[df['foto_mes'] == mes_train]
predecir = df[df['foto_mes'] == mes_test]

# Separar las características y el target
X = train.drop(columns=['clase_ternaria',"clase_binaria"])  # Variables predictoras
y = train['clase_binaria']  # Variable objetivo

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=semillas[0])

In [20]:
sss_opt = ShuffleSplit(n_splits=5, test_size=0.3, random_state=semillas[1])

def objective(trial, X, y, sss):
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 2, 20)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 150)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 150)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 150)

  def train_and_evaluate(train_index, test_index, X, y):
    m = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_leaf_nodes=max_leaf_nodes,
        random_state=semillas[0],
    )
    m.fit(X.iloc[train_index],y.iloc[train_index])
    ganancia_value = ganancia(m, X.iloc[test_index], y.iloc[test_index], prop=0.3)
    return ganancia_value

  results = Parallel(n_jobs=-1)(
      delayed(train_and_evaluate)(train_index, test_index, X, y)
      for train_index, test_index in sss.split(X)
  )

  return np.mean(results)

#storage_name = "sqlite:///G:/DM EyF/optimization_tree.db"
study_name = "exp_101_decision-tree-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    #storage=storage_name,
    #load_if_exists=True,
)

[I 2024-09-12 00:04:32,587] A new study created in memory with name: exp_101_decision-tree-opt


In [21]:
study.optimize(lambda trial: objective(trial, X, y, sss_opt), n_trials=50)

[I 2024-09-12 00:04:45,586] Trial 0 finished with value: 281372000.0 and parameters: {'criterion': 'gini', 'max_depth': 16, 'min_samples_split': 114, 'min_samples_leaf': 45, 'max_leaf_nodes': 133}. Best is trial 0 with value: 281372000.0.
[I 2024-09-12 00:04:54,952] Trial 1 finished with value: 313282666.6666667 and parameters: {'criterion': 'entropy', 'max_depth': 19, 'min_samples_split': 88, 'min_samples_leaf': 148, 'max_leaf_nodes': 64}. Best is trial 1 with value: 313282666.6666667.
[I 2024-09-12 00:05:01,874] Trial 2 finished with value: 318976000.00000006 and parameters: {'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 58, 'min_samples_leaf': 37, 'max_leaf_nodes': 28}. Best is trial 2 with value: 318976000.00000006.
[I 2024-09-12 00:05:04,924] Trial 3 finished with value: 242942000.0 and parameters: {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 4, 'min_samples_leaf': 48, 'max_leaf_nodes': 28}. Best is trial 2 with value: 318976000.00000006.
[I 2024-09-1

In [22]:
best_trial = study.best_trial
best_model_params = best_trial.params
print("Mejor modelo:", best_model_params)

Mejor modelo: {'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 142, 'min_samples_leaf': 9, 'max_leaf_nodes': 41}


In [31]:
# Definir conjunto de entrenamiento y donde voy a predecir
train = df[df['foto_mes'] == 202104].copy()
predecir = df[df['foto_mes'] == 202106].copy()

modelo = DecisionTreeClassifier(
    criterion= 'entropy',       # criterio
    max_depth=12,            # Profundidad máxima de cada árbol
    min_samples_split=142,   
    min_samples_leaf=9,
    max_leaf_nodes= 41,    
    random_state=semillas[0]
)

# Entrenar el modelo
X_train = train.drop(columns=['clase_binaria', 'clase_ternaria'])
y_train = train['clase_binaria']
modelo.fit(X_train, y_train)

# Aplicar el modelo a los datos nuevos
X_predecir = predecir.drop(columns=['clase_binaria', 'clase_ternaria'])
y_pred = modelo.predict_proba(X_predecir)

# Crear un DataFrame con las probabilidades
prediccion_df = pd.DataFrame(y_pred, columns=modelo.classes_)

# Agregar la probabilidad de clase '1' (BAJA) al dataframe de aplicación
predecir['prob_baja'] = prediccion_df[1]  # '1' es la clase correspondiente a BAJA

# Crear la columna 'Predicted' donde la probabilidad de 'BAJA' sea mayor a 1/40
predecir['Predicted'] = (predecir['prob_baja'] > 1 / 40).astype(int)

# Generar el archivo para Kaggle
# Crear el directorio si no existe
os.makedirs("./exp/KA2001", exist_ok=True)

# Seleccionar solo los campos necesarios para Kaggle
kaggle_submission = predecir[['numero_de_cliente', 'Predicted']]

# Guardar el archivo
kaggle_submission.to_csv("./exp/KA2001/K101_003.csv", index=False, sep=",")

In [32]:
predecir[predecir['Predicted']==1]

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,clase_binaria,prob_baja,Predicted
57,252966056,202106,1,0,0,56,218,1924.55,31626.42,4114.42,...,0.00,8661.0,27764.42,8.0,0.0,5313.69,NaN,1,0.027088,1
87,255614475,202106,1,0,0,65,283,5732.54,41402.14,746.84,...,0.00,6128.0,12695.07,7.0,0.0,656.88,NaN,1,0.085811,1
158,259824701,202106,1,0,0,56,79,1016.80,-1740.48,737.50,...,0.00,2347.0,5728.81,4.0,0.0,480.93,NaN,1,0.027088,1
218,264937841,202106,1,0,0,49,159,-74.32,7306.47,3147.26,...,0.00,4816.0,1489.47,4.0,0.0,7002.81,NaN,1,0.067002,1
230,268128619,202106,1,0,0,65,288,2207.76,41356.07,715.27,...,0.00,5397.0,3775.87,1.0,0.0,1184.73,NaN,1,0.321818,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164587,322502746,202106,1,0,0,47,204,855.38,3965.81,436.91,...,0.00,5963.0,19093.07,6.0,0.0,2557.14,NaN,1,0.074205,1
164689,332435849,202106,1,0,0,57,260,15492.70,139250.38,3202.23,...,0.00,7918.0,557.73,1.0,0.0,11.73,NaN,1,0.067002,1
164695,332496302,202106,1,0,0,42,177,-498.60,-2193.25,797.59,...,34.39,5344.0,36799.83,19.0,0.0,6439.77,NaN,1,0.067002,1
164749,339121872,202106,1,0,0,55,378,547.26,28134.24,27.84,...,0.00,9365.0,2457.43,1.0,0.0,117.30,NaN,1,0.085811,1
